In [ ]:
%pip install tensorflow keras pillow

### Importing necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cv2
import random
import os
from os import listdir
from PIL import Image
import tensorflow as tf
from keras.preprocessing import image
from tensorflow.keras.utils import img_to_array, array_to_img
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dropout, Dense
from sklearn.model_selection import train_test_split
from keras.models import model_from_json
from keras.utils import to_categorical

In [ ]:
print(tf.__version__)

### Defining the path of dataset directory

In [3]:
dataset_path = "D:\plant_disease_project\Groundnut_Leaf_dataset"
labels = os.listdir(dataset_path)

print(labels)

### Visualizing the images and Resize images

In [ ]:
# # Plotting 12 images to check dataset

# plt.figure(figsize=(12, 12))
# dataset_path = (
#     r"D:\plant_disease_project\Groundnut_Leaf_dataset\train\early_leaf_spot_1"
# )

# for i in range(1, 17):
#     plt.subplot(4, 4, i)
#     plt.tight_layout()
#     rand_img = imread(
#         dataset_path + "/" + random.choice(sorted(os.listdir(dataset_path)))
#     )
#     plt.imshow(rand_img)
#     plt.xlabel(rand_img.shape[1], fontsize=10)  # width of image
#     plt.ylabel(rand_img.shape[0], fontsize=10)  # height of image

### Convert the images into a Numpy array and normalize them

In [12]:
# Converting Images to array
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None:
            image = cv2.resize(image, (256, 256))
            return img_to_array(image)
        else:
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [16]:
root_dir = listdir(dataset_path)
image_list, label_list = [], []
all_labels = [
    "early_leaf_spot_1",
    "early_rust_1",
    "healthy_leaf_1",
    "late_leaf_spot_1",
    "nutrition_deficiency_1",
    "rust_1",
]
binary_labels = [0, 1, 2, 3, 4, 5]
temp = -1

# Reading and converting image to numpy array

for directory in root_dir:
    plant_image_list = listdir(f"{dataset_path}/{directory}")
    temp += 1
    for files in plant_image_list:
        image_path = f"{dataset_path}/{directory}/{files}"
        image_list.append(convert_image_to_array(image_path))
        label_list.append(binary_labels[temp])

### Visualize the class count and Check for class imbalance

In [ ]:
# Visualize the number of classes count

label_counts = pd.DataFrame(label_list).value_counts()
label_counts.head()

In [ ]:
# Next we will observe the shape of the image.

image_list[0].shape

In [ ]:
# Checking the total number of the images which is the length of the labels list.

label_list = np.array(label_list)
label_list.shape

### Splitting the dataset into train, validate and test sets

In [23]:
x_train, x_test, y_train, y_test = train_test_split(
    image_list, label_list, test_size=0.2, random_state=10
)

In [24]:
# Now we will normalize the dataset of our images. As pixel values ranges from 0 to 255 so we will divide each image pixel with 255 to normalize the dataset.

x_train = np.array(x_train, dtype=np.float16) / 225.0
x_test = np.array(x_test, dtype=np.float16) / 225.0
x_train = x_train.reshape(-1, 256, 256, 3)
x_test = x_test.reshape(-1, 256, 256, 3)

### Performing one-hot encoding on target variable

In [25]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

### Creating the model architecture, compile the model and then fit it using the training data

In [ ]:
model = Sequential()
model.add(
    Conv2D(32, (3, 3), padding="same", input_shape=(256, 256, 3), activation="relu")
)
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(16, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(8, activation="relu"))
model.add(Dense(3, activation="softmax"))
model.summary()

In [27]:
model.compile(
    loss="categorical_crossentropy", optimizer=Adam(0.0001), metrics=["accuracy"]
)

In [28]:
# Splitting the training data set into training and validation data sets

x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.2, random_state=10
)

In [ ]:
# Training the model

epochs = 50
batch_size = 128
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_val, y_val),
)

In [36]:
model.save("C:\Plant-Disease-Detection\Model\plant_disease_model.h5")

### Plot the accuracy and loss against each epoch

In [ ]:
# Plot the training history

plt.figure(figsize=(12, 5))
plt.plot(history.history["accuracy"], color="r")
plt.plot(history.history["val_accuracy"], color="b")
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epochs")
plt.legend(["train", "val"])

plt.show()

In [ ]:
print("Calculating model accuracy")

scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1] * 100}")

### Make predictions on testing data

In [ ]:
y_pred = model.predict(x_test)

### Visualizing the original and predicted labels for the test images

In [ ]:
# Plotting image to compare

img = array_to_img(x_test[11])
img

In [ ]:
# Finding max value from predition list and comaparing original value vs predicted

print("Originally : ", all_labels[np.argmax(y_test[11])])
print("Predicted : ", all_labels[np.argmax(y_pred[4])])
print(y_pred[2])

In [ ]:
for i in range(50):
    print(all_labels[np.argmax(y_test[i])], " ", all_labels[np.argmax(y_pred[1])])